In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import numpy as np
import tensorflow as tf
import requests
from PIL import Image

## Download human-readable labels for ImageNet.

In [ ]:
response = requests.get("https://git.io/JJkYN")
labels = response.text.split('\n')

## Load models

In [ ]:
inception_net = tf.keras.applications.InceptionV3()
mobile_net = tf.keras.applications.MobileNetV2()

## Create funtion to Mobile net and inception net classifier

In [ ]:
def mobilenet_classify(im):
  im = Image.fromarray(im.astype('uint8'), 'RGB')
  im = im.resize((224, 224))
  arr = np.array(im).reshape((-1, 224, 224, 3))
  arr = tf.keras.applications.mobilenet.preprocess_input(arr)
  prediction = mobile_net.predict(arr).flatten()
  return {labels[i]: float(prediction[i]) for i in range (1000)}

In [ ]:
def classify_img(im):
  im = Image.fromarray(im.astype('uint8'), 'RGB')
  im = im.resize((299, 299))
  arr = np.array(im).reshape((-1, 299, 299, 3))
  arr = tf.keras.applications.inception_v3.preprocess_input(arr)
  prediction = inception_net.predict(arr).flatten()
  return {labels[i]: float(prediction[i]) for i in range (1000)}

## Gradio

### Gradio Inputs and outpus

In [ ]:
image = gr.inputs.Image()
label = gr.outputs.Label(num_top_classes=3)
sample_images = [
                 ["monkey.jpg"],
                 ["sailboat.jpg"],
                 ["bicycle.jpg"],
                 ["download.jpg"],
                 ["fox.jpg"]
]

In [ ]:
##Gradio interfaces
gr.Interface([classify_img, mobilenet_classify], inputs=image,outputs=label,
             capture_session =True, interpretation = "default").launch()